In [1]:
import rasterio

tif_path =r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\ERA5_30m_stack_fixed.tif"

with rasterio.open(tif_path) as src:
    print("NoData value:", src.nodata)

NoData value: nan


In [2]:
import rasterio
import numpy as np

tif_path = r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\ERA5_30m_stack_fixed.tif"

with rasterio.open(tif_path) as src:
    data = src.read(masked=True)  # masked=True replaces NoData with masked entries
    total_pixels = data.size
    missing_pixels = np.ma.count_masked(data)
    missing_percentage = (missing_pixels / total_pixels) * 100

print(f"Total pixels: {total_pixels:,}")
print(f"Missing pixels: {missing_pixels:,}")
print(f"Missing percentage: {missing_percentage:.2f}%")


Total pixels: 809,710,080
Missing pixels: 23,487,325
Missing percentage: 2.90%


In [ ]:
import rasterio
import numpy as np
from rasterio.fill import fillnodata
from pathlib import Path

# Input and output paths
in_tif = Path(r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\ERA5_30m_stack_fixed.tif")
out_tif = in_tif.with_name(in_tif.stem + "_filled.tif")

with rasterio.open(in_tif) as src:
    profile = src.profile.copy()
    profile.update(dtype=rasterio.float32, nodata=None, compress='LZW')

    # Read all bands
    data = src.read().astype("float32")  # shape: (bands, rows, cols)
    filled_data = np.empty_like(data)

    # Loop through each band
    for i in range(src.count):
        arr = data[i]
        mask = np.isnan(arr) | (arr == src.nodatavals[i] if src.nodatavals[i] is not None else False)

        # Fill nodata values
        filled = fillnodata(arr, mask=mask, max_search_distance=100, smoothing_iterations=0)
        filled_data[i] = filled

    # Save the fixed raster
    with rasterio.open(out_tif, "w", **profile) as dst:
        dst.write(filled_data)

print("✅ Multi-band filled raster saved at:", out_tif)
